In [1]:
import os
import requests
import bs4
import openai

In [2]:
# Read API Key
with open('API_KEY.txt') as f:
    api_key = f.readlines()[0]

# Set API Key
openai.api_key = api_key

## News Translation & Summarization

Summarize news from any country.

Steps:

```
1. Accept user's request for news of a creain country
2. Scrape news from a newspaper for that country
3. Send this data to OpenAI API to Translate & Summarize
```

NOTE: OpenAI inherently is not trained on Translation tasks. Also, for other languages -> English, this works better as compared to other way around.

In [3]:
# Country: (URL, TAG)
country_newspapers = {
    "Spain": ("https://elpais.com/", '.c_t'),
    "France": ("https://www.lemonde.fr/", '.article__title'),
    "India": ("https://www.thehindu.com/hindi/", '.title')
}

## Send Translation Request to OpenAI

In [4]:
def create_prompt():
    country = input("What country would you like a news summary for? ")
    try:
        url, tag = country_newspapers[country]
    except:
        print("Sorry, Country not supported")
        return
    
    results = requests.get(url)
    # Create Soup
    soup = bs4.BeautifulSoup(results.text, 'lxml')
    country_headlines = ''
    
    # Top 5 News Headlines
    for item in soup.select(str(tag))[:5]:
        country_headlines += item.getText() + "\n"
    
    # OpenAI Prompt
    prompt = "Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone.\n"
    
    return prompt + country_headlines

In [5]:
prompt = create_prompt()

What country would you like a news summary for?  India


In [6]:
print(prompt)

Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone.
 बैंकिंग संकट के सबक: सिलीकॉन वैली बैंक प्रकरण 
 तीखे विभाजन: “ऑकस” त्रिपक्षीय रक्षा समझौते के निहितार्थ 
 स्पिन की दिक्कत : भारतीय टीम और वर्ल्ड टेस्ट चैंपियनशिप 
 सदन की कार्रवाईः संसद में गतिरोध 

अंतहीन देरी: राज्यपालों का आचरण और विधायी एजेंडा 



In [7]:
# Send to OpenAI
response = openai.Completion.create(
    model='text-davinci-003',
    prompt=prompt,
    max_tokens=512,
    temperature=0.1,
    top_p=1.0,
)

In [9]:
print(response['choices'][0]['text'])


Language: Hindi

Translation: The banking crisis has been a major issue, with the Silicone Valley Bank case being a prime example. The "Oaks" trilateral defense agreement was signed in order to address the issue. The Indian team and the World Test Championship have been facing difficulties. There has been a slowdown in the Parliament. The behavior of the state governors and the functioning of the legislative agency has been delayed.
